# Topic modelling
## Unsupervised learning of topics in a text
### using Latent Dirchlet Allocation (via sklearn)
Topic modelling can be thought of as dimensionality reduction:  
Documents are represented as sets of topics  
Each topic has a weight

In [1]:
import re
import pandas as pd
import sklearn
import csv
import nltk
nltk.download('punkt')
import string
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# use CountVectorizer to turn the docs into vectors
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [3]:
# create the stemmer
stemmer = SnowballStemmer('english')

In [4]:
def read_in_csv(csv_file):
    with open(csv_file, 'r', encoding='utf-8') as fp:
        reader = csv.reader(fp, delimiter=',', quotechar='"')
        data_read = [row for row in reader]
    return data_read

In [5]:
stopwords_file_path = "stopwords.csv"

def get_stopwords(path=stopwords_file_path):
    stopwords = read_in_csv(path)
    stopwords = [word[0] for word in stopwords]
    stemmed_stopwords = [stemmer.stem(word) for word in stopwords]
    stopwords = stopwords + stemmed_stopwords
    return stopwords

In [6]:
# read in our data
stopwords = get_stopwords(stopwords_file_path)
bbc_dataset = "bbc-text.csv"

# helper functions
def tokenize_and_stem(sentence):
    tokens = nltk.word_tokenize(sentence)
    filtered_tokens = [t for t in tokens if t not in stopwords and t not in string.punctuation and re.search('[a-zA-Z]', t)]
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

### We’ll use a public dataset from the BBC comprised of 2,225 articles  
Each labeled under one of 5 categories: business, entertainment, politics, sport or tech

In [7]:
# turn the documents into vectors
def create_count_vectorizer(documents):
    count_vectorizer = CountVectorizer(stop_words=stopwords, tokenizer=tokenize_and_stem, max_features=1500)
    data = count_vectorizer.fit_transform(documents)
    return (count_vectorizer, data)

In [8]:
# remove unwanted characters (keep just words and spaces)
def clean_data(df):
    df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
    df['text'] = df['text'].apply(lambda x: re.sub(r'\d', '', x))
    return df

In [9]:
# create the LDA model (note that usually num_topics is unknown)
def create_and_fit_lda(data, num_topics):
    lda = LDA(n_components=num_topics, n_jobs=-1)
    lda.fit(data)
    return lda

In [10]:
# identify & print the most common topic words
def get_most_common_words_for_topics(model, vectorizer, n_top_words):
    words = vectorizer.get_feature_names()
    word_dict = {}
    for topic_index, topic in enumerate(model.components_):
        this_topic_words = [words[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        word_dict[topic_index] = this_topic_words
    return word_dict

def print_topic_words(word_dict):
    for key in word_dict.keys():
        print(f"Topic {key}")
        print("\t", word_dict[key])

In [11]:
# read in the data, clean it, get text
df = pd.read_csv(bbc_dataset)
df = clean_data(df)
documents = df['text']

# set number of topics (note that usually this is unknown)
number_topics = 5

### Step 1: Extract one of the categories  
Select a particular category from the dataframe, e.g. tech

In [12]:
df = df.loc[df['category']=='tech']

In [13]:
documents = df['text']

In [14]:
# check the output
df

,category,text
0,tech,tv future in the hands of viewers with home th...
19,tech,games maker fights for survival one of britain...
20,tech,security warning over fbi virus the us feder...
21,tech,halo heralds traffic explosion the growing po...
24,tech,mobile audio enters new dimension as mobile ph...
...,...,...
2204,tech,argonaut founder rebuilds empire jez san the ...
2207,tech,california sets fines for spyware the makers o...
2213,tech,progress on new internet domains by early the...
2215,tech,junk e mails on relentless rise spam traffic i...


In [15]:
# create vectorizer & model
(vectorizer, data) = create_count_vectorizer(documents)
lda = create_and_fit_lda(data, number_topics)

### Step 2: Inspect the results  
Are they coherent? Do they seem to be different topics?

In [16]:
# inspect the contents of the topics
topic_words = get_most_common_words_for_topics(lda, vectorizer, 10)
print_topic_words(topic_words)

Topic 0
	 ['mobil', 'phone', 'use', 'peopl', 'technolog', 'make', 'servic', 'camera', 'network', 'year']
Topic 1
	 ['game', 'search', 'play', 'peopl', 'time', 'get', 'use', 'world', 'year', 'site']
Topic 2
	 ['music', 'peopl', 'servic', 'file', 'appl', 'digit', 'player', 'net', 'use', 'gadget']
Topic 3
	 ['technolog', 'tv', 'digit', 'high', 'video', 'peopl', 'year', 'broadband', 'show', 'content']
Topic 4
	 ['softwar', 'use', 'secur', 'virus', 'mail', 'program', 'firm', 'user', 'e', 'microsoft']


In [ ]:
def test_new_example(lda,vect,example):
    vectorized=vect.transform([example])
    topic=lda.transform(vectorized)
    print(topic)
    return topic

### Step 3: Try a different category  
Select a different category from the dataframe, e.g. sport

### Step 4: Inspect the results  
Are they coherent? Do they seem to be different topics?

### Step 5: Try different values of N  
Return to Step 1 and repeat the process with a different number of topics